### Environment
Run this under tf-cpu3 envr, although probably no longer needed. 

In [3]:
import quopri
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Read files and extract elements

Load from downloaded html page with dynamically loaded contents, since the raw HTML didn't include the actual listings. 

In [4]:
# Open the file
with open("tennis_listing.mhtml", "r", encoding='utf-8') as f:
    content = f.read()

# Decode the Quoted-Printable encoding
decoded_content = quopri.decodestring(content).decode('utf-8')

# Parse the page content
soup = BeautifulSoup(decoded_content, 'lxml')

# Find the table in the page content
table = soup.find('table')

# Get all rows in the table
rows = table.find_all('tr')

# Get the column headers
headers = [th.text.strip() for th in rows[0].find_all('th')]

# Initialize empty list to hold the row data
data = []

# Iterate over the rows, starting from the second one (as the first one is the header)
for row in rows[1:]:
    # Get the tds in the row
    tds = row.find_all('td')

    # Extract the desired data from the tds based on data-info attribute
    location = next((td.text for td in tds if td.get('data-info') == 'Location'), '')
    public_hours = next((td.text for td in tds if td.get('data-info') == 'Public Hours'), '')
    winter_play = next((td.get_text().strip() for td in tds if td.get('data-info') == 'WinterPlay'), 'NA')
    type = next((td.get_text().strip() for td in tds if td.get('data-info') == 'Type'), '')
    lights = next((td.get_text().strip() for td in tds if td.get('data-info') == 'Lights'), '')
    courts = next((td.get_text().strip() for td in tds if td.get('data-info') == 'Courts'), '')

    # Append the data to the list
    data.append([location, public_hours, winter_play, type, lights, courts])

# Create a DataFrame with the data
df = pd.DataFrame(data, columns=['location', 'public_hours', 'winter_play', 'type', 'lights', 'courts'])

In [5]:
# separate df['location'] into 2 columns based, putting everything before 'Club:' into df['club'] and everything after into df['additional'] 
df[['place','additional']] = df['location'].str.split('Club:',expand=True)

In [6]:
# use geocoders to get long and lat for each place
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="geoapi_tenniscourts_toronto_test")

# Initialize lists to store the latitudes and longitudes
latitudes = []
longitudes = []

# Iterate over the places
for place in df['place']:
    # Add "Toronto" to the place name to improve geocoding accuracy
    location = geolocator.geocode(f"{place}, Toronto")

    # Check if the geocoding was successful
    if location is not None:
        # If it was, append the latitude and longitude to the lists
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)
    else:
        # If it wasn't, append a missing value
        latitudes.append(None)
        longitudes.append(None)

    # Sleep for 1 second to avoid hitting the rate limit
    time.sleep(1)

# Add the latitudes and longitudes to the dataframe
df['latitude'] = latitudes
df['longitude'] = longitudes


KeyboardInterrupt: 

In [ ]:
# get subset of df where latitude and longitude are null
df_null = df[df[['latitude', 'longitude']].isnull().any(axis=1)]
# replace df_null['place'] with the part of df_null['location'] before ' Book Online'
df_null['place'] = df_null['place'].str.split(' Book Online',expand=True)[0]
# replace df_null['place'] with the part of df_null['place'] before ' -'
df_null['place'] = df_null['place'].str.split(' -',expand=True)[0]
# replace df_null['place'] with the part of df_null['place'] before comma ','
df_null['place'] = df_null['place'].str.split(',',expand=True)[0]
df_null

In [ ]:
# modify code to get long and lat for places missed the 1st time.

from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="geoapi_tenniscourts_toronto_test")

# Initialize lists to store the latitudes and longitudes
latitudes = []
longitudes = []

# Iterate over the places
for place in df_null['place']:
    # Add "Toronto" to the place name to improve geocoding accuracy
    location = geolocator.geocode(f"{place}, Toronto")

    # Check if the geocoding was successful
    if location is not None:
        # If it was, append the latitude and longitude to the lists
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)
    else:
        # If it wasn't, append a missing value
        latitudes.append(None)
        longitudes.append(None)

    # Sleep for 1 second to avoid hitting the rate limit
    time.sleep(1)

# Add the latitudes and longitudes to the dataframe
df_null['latitude'] = latitudes
df_null['longitude'] = longitudes

df_null

In [ ]:
## Combine both dataframes

# load tennis_courts_toronto_cleaned.csv to df_manual
df_manual = pd.read_csv('tennis_courts_toronto_null.csv')

# concat df_manual with df and remove duplicates based on location and keep the one from df_mannual
df_merged = pd.concat([df, df_manual]).drop_duplicates(subset=['location'], keep='last')
df_merged

# count null values in df_merged for latitude and longitude
df_merged.shape

The `df_manual` file, loaded from the csv file, involved some mannual work to cover the courts that was missed by the geocoder package. The results are merged, and saved to an external csv file in the code block below. 

In [ ]:
# save df_merged to csv
# df_merged.to_csv('tennis_courts_toronto_full_cleaned.csv', index=False)

# load df_merged from csv
df_complete = pd.read_csv('tennis_courts_toronto_full_cleaned.csv')
df_complete

## Plot map and show results

In [ ]:
# save df_merged to csv
# df_merged.to_csv('tennis_courts_toronto_full_cleaned.csv', index=False)

# load df_merged from csv
df_complete = pd.read_csv('tennis_courts_toronto_full_cleaned.csv')
df_complete


import folium

# Create a map centered at an approximate location of Toronto
map_toronto = folium.Map(location=[43.70, -79.42], zoom_start=11)

# filter
df_complete = df_complete[df_complete['type'] == 'Public']
# df_complete = df_complete[df_complete['lights'] == 'Yes']

# Add a marker for each tennis club to the map
for index, row in df_complete.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['place'],  # this will show the name of the place when you click on the marker
    ).add_to(map_toronto)

# Display the map
map_toronto
